In [91]:
import pandas as pd
import numpy as np

import warnings
from scipy import integrate

In [4]:
#################################### DADOS ###################################

filepath = r"C:\Users\user\Projetos\densities4risk\data\processed\database.parquet"
df = pd.read_parquet(filepath)
TICKER = 'BVSP'
df_bvsp = df[df["ticker"]==TICKER].loc[:,["datetime", "open", "high", "low", "close"]].dropna().drop_duplicates()
df_bvsp.set_index("datetime", inplace=True)

start_time = '10:00'
end_time = '17:00'
freq = '5T'  # 5 minutes

df_bvsp = df[df["ticker"]==TICKER].loc[:,["datetime", "open", "high", "low", "close"]].dropna().drop_duplicates()
df_bvsp.set_index("datetime", inplace=True)

# Create a complete time index for each day
def complete_day(group):
    day = group.index[0].date()
    full_index = pd.date_range(
        f"{day} {start_time}",
        f"{day} {end_time}",
        freq=freq
    )
    return group.reindex(full_index)

# Apply to each day
df_complete = df_bvsp.groupby(df_bvsp.index.date, group_keys=False).apply(complete_day)
# Reset index and rename column
df_complete = df_complete.reset_index().rename(columns={'index': 'datetime'})
df_complete.interpolate(method="linear", inplace=True)

df_complete["R_t"] = df_complete["close"].pct_change()
df_complete["r_t"] = np.log(df_complete["close"]) - np.log(df_complete["close"].shift(1))

C:\Users\user\AppData\Local\Temp\ipykernel_3460\127940432.py:19: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(


In [174]:
# example
l_inf = 0
l_sup = 2
n = 512
prob = 0.5

dSup = np.linspace(l_inf, l_sup, n) # support (grid) for Density domain
dens = np.full(n, prob) # density values on dSup - must be strictly positive (otherwise will truncate) and integrate to 1

In [175]:
# Function to convert densities to log quantile density functions

if np.any(dens)<0:
    raise Exception("There are negative density values")
if np.abs(np.trapezoid(dens, dSup) - 1) > 1e-5:
    warnings.warn("Density does not integrate to 1 with tolerance of 1e-5 - renormalizing now.")
    dens = dens/np.trapezoid(dens, dSup)
if np.any(dens==0):  # FUNÇÃO NÃO CONSIDERA POSSIBILIDADE DE ZERO ENTRE LIMITES DE (SUPORTE>O)
    warnings.warn("There are some zero density values - truncating support grid so all are positive.")
    lower_bound = np.min(np.where(dens>0))
    upper_bound = np.max(np.where(dens>0))
    dSup = dSup[lower_bound : upper_bound]
    dens = dens/np.trapezoid(dens, dSup) # ENTENDER POR QUE ISSO FUNCIONA
N = len(dSup)

In [176]:
lqdSup = None #support of length M for lqd domain - must begin at 0 and end at 1; (default: seq(0, 1, length(dSup)))
lqdSup = np.linspace(0,10,11)
lqdSup

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [177]:
SUP = np.array([0,1])

In [178]:
if lqdSup is None:
    lqdSup = np.linspace(0,1,N)
elif not (np.array([lqdSup.min(),lqdSup.max()]) == SUP).all():
    warnings.warn(("Problem with support of the LQD domain's boundaries - resetting to default."))
    lqdSup = np.linspace(0,1,N)

C:\Users\user\AppData\Local\Temp\ipykernel_3460\3479929737.py:4: UserWarning: Problem with support of the LQD domain's boundaries - resetting to default.
  warnings.warn(("Problem with support of the LQD domain's boundaries - resetting to default."))


In [179]:
t0 = dSup[0] # value in dSup for which the cdf value c is retained, i.e. c = F(t0) (default: dSup[0])

In [180]:
if not (t0 in dSup):
    warnings.warn("t0 is not a value in dSup - resetting to closest value")
    t0 = dSup[np.argmin((abs(dSup-t0)))]

In [181]:
M = len(lqdSup)
c_ind = np.where(dSup == t0)[0][0]

In [182]:
# Get CDF and lqd on temporary grid, compute c
tmp = integrate.cumulative_trapezoid(dens, dSup, initial=0)
c = tmp[c_ind]

In [ ]:
N_tmp = len(tmp)

# ---- Left duplicates (forward scan)
first_half = tmp[:N_tmp//2]
_, indL = np.unique(first_half, return_index=True)
maskL = np.ones(len(first_half), dtype=bool)
maskL[np.setdiff1d(np.arange(len(first_half)), indL)] = False  # mark duplicates as False

# ---- Right duplicates (reverse scan)
second_half = tmp[N_tmp//2:]
_, indR_rev = np.unique(second_half[::-1], return_index=True)
maskR = np.ones(len(second_half), dtype=bool)
maskR[np.setdiff1d(np.arange(len(second_half)), len(second_half) - 1 - indR_rev)] = False

mask = np.concatenate([maskL, maskR])
qtemp = tmp[mask]

lqd_temp = -np.log(dens[mask])

In [220]:
# Interpolate lqdSup, keeping track of Inf values at boundary, then compute c
lqd = np.full(M, 0)

if np.any(np.isfinite([lqd_temp[0], lqd_temp[N-1]])):
    tmpInd = np.arange(1, N)
    Ind = np.arange(1, M)
    if lqd_temp[0] == np.inf:
        lqd[0] = np.inf
        tmpInd = tmpInd[-1]
        Ind = Ind[-1]
    if lqd_temp[-1] == np.inf:
        lqd[M] = np.inf
        tmpInd = tmpInd[-len(tmpInd)]
        Ind = Ind[-len(Ind)]

    interp_values = np.interp(
        lqdSup[Ind-1],                 # xout
        qtemp[tmpInd-1],               # x
        lqd_temp[tmpInd-1],            # y
        left=lqd_temp[tmpInd-1][0],    # rule = 2 → use endpoint values
        right=lqd_temp[tmpInd-1][-1]
    )

    lqd[Ind] = interp_values
else:
    interp_values = np.interp(
        lqdSup,
        qtemp,
        lqd_temp,
        left=lqd_temp[0],    # rule = 2 → use endpoint values
        right=lqd_temp[-1]
    )
    lqd = interp_values

In [223]:
['lqdSup', lqdSup, 'lqd', lqd, 'c', c]

['lqdSup',
 array([0.        , 0.00195695, 0.00391389, 0.00587084, 0.00782779,
        0.00978474, 0.01174168, 0.01369863, 0.01565558, 0.01761252,
        0.01956947, 0.02152642, 0.02348337, 0.02544031, 0.02739726,
        0.02935421, 0.03131115, 0.0332681 , 0.03522505, 0.037182  ,
        0.03913894, 0.04109589, 0.04305284, 0.04500978, 0.04696673,
        0.04892368, 0.05088063, 0.05283757, 0.05479452, 0.05675147,
        0.05870841, 0.06066536, 0.06262231, 0.06457926, 0.0665362 ,
        0.06849315, 0.0704501 , 0.07240705, 0.07436399, 0.07632094,
        0.07827789, 0.08023483, 0.08219178, 0.08414873, 0.08610568,
        0.08806262, 0.09001957, 0.09197652, 0.09393346, 0.09589041,
        0.09784736, 0.09980431, 0.10176125, 0.1037182 , 0.10567515,
        0.10763209, 0.10958904, 0.11154599, 0.11350294, 0.11545988,
        0.11741683, 0.11937378, 0.12133072, 0.12328767, 0.12524462,
        0.12720157, 0.12915851, 0.13111546, 0.13307241, 0.13502935,
        0.1369863 , 0.13894325, 0.140

In [ ]:
import numpy as np
import warnings
from scipy import integrate

def T_q(
        dens: np.array, 
        dens_support: np.array, 
        lqd_support: np.array = None,
        t0 : int = None
        ) -> list[np.array, np.array, np.float64]:
    """Function for converting densities to log quantile density functions

    Args:
        dens (np.array): density values on dSup - must be strictly positive (otherwise will truncate) and integrate to 1.
        dens_support (np.array): support (grid) for Density domain.
        lqd_support (np.array, optional): support of length M for lqd domain - must begin at 0 and end at 1. Defaults to None.
        t0 (int, optional): value in dSup for which the cdf value c is retained, i.e. c = F(t0). Defaults to None.

    Returns:
    list[np.array, np.array, np.float64]:
        lqd_support,
        lqd,
        c
    """
    if np.any(dens)<0:
        raise Exception("There are negative density values")
    if np.abs(np.trapezoid(dens, dens_support) - 1) > 1e-5:
        warnings.warn("Density does not integrate to 1 with tolerance of 1e-5 - renormalizing now.")
        dens = dens/np.trapezoid(dens, dens_support)
    if np.any(dens==0):  # FUNÇÃO NÃO CONSIDERA POSSIBILIDADE DE ZERO ENTRE LIMITES DE (SUPORTE>O)
        warnings.warn("There are some zero density values - truncating support grid so all are positive.")
        lower_bound = np.min(np.where(dens>0))
        upper_bound = np.max(np.where(dens>0))
        dens_support = dens_support[lower_bound : upper_bound]
        dens = dens/np.trapezoid(dens, dens_support) # ENTENDER POR QUE ISSO FUNCIONA
        
    N = len(dens_support)

    if lqd_support is None:
        lqd_support = np.linspace(0,1,N)
    elif not (np.array([lqd_support.min(),lqd_support.max()]) == np.array([0,1])).all():
        warnings.warn(("Problem with support of the LQD domain's boundaries - resetting to default."))
        lqd_support = np.linspace(0,1,N)
        
    if not (t0 in dens_support):
        warnings.warn("t0 is not a value in dSup - resetting to closest value")
        t0 = dens_support[np.argmin((abs(dens_support-t0)))]
    M = len(lqd_support)
    c_ind = np.where(dens_support == t0)[0][0]

    # Get CDF and lqd on temporary grid, compute c
    tmp = integrate.cumulative_trapezoid(dens, dens_support, initial=0)
    c = tmp[c_ind]

    N_tmp = len(tmp)

    # ---- Left duplicates (forward scan)
    first_half = tmp[:N_tmp//2]
    _, indL = np.unique(first_half, return_index=True)
    maskL = np.ones(len(first_half), dtype=bool)
    maskL[np.setdiff1d(np.arange(len(first_half)), indL)] = False  # mark duplicates as False

    # ---- Right duplicates (reverse scan)
    second_half = tmp[N_tmp//2:]
    _, indR_rev = np.unique(second_half[::-1], return_index=True)
    maskR = np.ones(len(second_half), dtype=bool)
    maskR[np.setdiff1d(np.arange(len(second_half)), len(second_half) - 1 - indR_rev)] = False

    mask = np.concatenate([maskL, maskR])
    qtemp = tmp[mask]

    lqd_temp = -np.log(dens[mask])

    # Interpolate lqd_support, keeping track of Inf values at boundary, then compute c
    lqd = np.full(M, 0)

    if np.any(np.isfinite([lqd_temp[0], lqd_temp[N-1]])):
        tmpInd = np.arange(1, N)
        Ind = np.arange(1, M)
        if lqd_temp[0] == np.inf:
            lqd[0] = np.inf
            tmpInd = tmpInd[-1]
            Ind = Ind[-1]
        if lqd_temp[-1] == np.inf:
            lqd[M] = np.inf
            tmpInd = tmpInd[-len(tmpInd)]
            Ind = Ind[-len(Ind)]

        interp_values = np.interp(
            lqd_support[Ind-1],                 # xout
            qtemp[tmpInd-1],               # x
            lqd_temp[tmpInd-1],            # y
            left=lqd_temp[tmpInd-1][0],    # rule = 2 → use endpoint values
            right=lqd_temp[tmpInd-1][-1]
        )

        lqd[Ind] = interp_values
    else:
        interp_values = np.interp(
            lqd_support,
            qtemp,
            lqd_temp,
            left=lqd_temp[0],    # rule = 2 → use endpoint values
            right=lqd_temp[-1]
        )
        lqd = interp_values

    return [lqd_support, lqd, c]

In [228]:
c

np.float64(0.0)